In [1]:
try:
    with open('i.html') as fp:
        text = fp.read()
except UnicodeDecodeError as exc:
    print(exc)

'utf-8' codec can't decode byte 0x80 in position 1721: invalid start byte


In [2]:
with open('i.html', 'rb') as fp:
    octets = fp.read()

for i in range(1715, 1725):
    code = octets[i]
    octet = octets[i:i+1]
    char = octet.decode('cp1252')
    print(f'{code:02x} {char!r}')

0a '\n'
0a '\n'
3c '<'
21 '!'
2d '-'
2d '-'
80 '€'
80 '€'
80 '€'
80 '€'


In [3]:
with open('i.html', encoding='cp1252') as fp:
     text = fp.read()

In [23]:
import re
import html

hexagram_start = re.compile(r'<a name="\d{1,2}"></a>')
title_parts = re.compile(r'(\d{1,2})\. ([^\n]+)')
line_1 = re.compile(r'(Six|Nine) (at|in) the beginning means:')
line_2 = re.compile(r'(Six|Nine) in \b?the second place means:')
line_6 = re.compile(r'(Six|Nine) at the top means:')
chapters = hexagram_start.split(html.unescape(text))
for i, chapter in enumerate(chapters[1:], 1):
    verses = chapter.split('\n')
    hit = title_parts.search(verses[1])
    n, title = hit.groups(verses[1])
    title_wg, title_en = (s.strip() for s in title.split('/'))
    assert i == int(n)
    print(f'{n:2}\t{title_wg:10} {title_en}')
    line_matches = 0
    for verse in verses[2:]:
        hit1 = line_1.search(verse)
        if hit1 is not None:
            print('\t', hit1.group(1))
            line_matches += 1
            continue
        hit2 = line_2.search(verse)
        if hit2 is not None:
            print('\t', hit2.group(1))
            line_matches += 1
            continue
        hit6 = line_6.search(verse)
        if hit6 is not None:
            print('\t', hit6.group(1))
            line_matches += 1
            continue
    if line_matches != 3:
        print('!!! missing line text')

error: nothing to repeat at position 16